In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
from sklearn.preprocessing import StandardScaler 
from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

In [ ]:
btc = pd.read_csv('btc_prediction_datawranglingStage.csv')

In [ ]:
btc.info()

In [ ]:
target = btc['close'].values

In [ ]:
btc_features = btc.drop('Date', axis=1)

In [ ]:
scaler = MinMaxScaler(feature_range=(0, 1))
btc_scaled = scaler.fit_transform(btc_features)

In [ ]:
btc_scaled_df = pd.DataFrame(btc_scaled, columns=btc_features.columns)
btc_scaled_df['Date'] = btc['Date']
btc_scaled_df.set_index('Date', inplace=True)
print(btc_scaled_df.head())

In [ ]:
seq_len = 1
train_size = int(len(btc_scaled_df) * 0.9)
train_data = btc_scaled_df.iloc[:train_size]
test_data = btc_scaled_df.iloc[train_size:]

In [ ]:
def create_sequences(data, seq_len):
    if len(data) < seq_len:
        raise ValueError("Length of data is less than sequence length")
    X = []
    y = []
    for i in range(seq_len, len(data)):
        X.append(data.iloc[i-seq_len:i].values)
        y.append(data.iloc[i].values)
    X = np.array(X)
    y = np.array(y)
    return X, y

In [ ]:
input_cols = ['open', 'high', 'low', 'close',]
X_train, y_train = create_sequences(train_data[input_cols], seq_len)
X_test, y_test = create_sequences(test_data[input_cols], seq_len)

In [ ]:
X_train

In [ ]:
X_train.shape

In [ ]:
y_train.shape

In [ ]:
y_test

In [ ]:
model = Sequential()
model.add(LSTM(units=128, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(LSTM(128, return_sequences=True))
model.add(LSTM(64))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='Adam')

In [ ]:
model.fit(X_train, y_train, epochs=15, batch_size=32, verbose=1)

In [ ]:
test_loss = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', test_loss)

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(10,6))
plt.plot(y_test[:, 3], label='actual')
plt.plot(y_pred[:, 0], label='predicted')
plt.legend()
plt.show()